In [1]:
import numpy as np
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn


/home/volody/code/study-py/ts-pytorch/.venv/lib/python3.11/site-packages/lightning_utilities/core/imports.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:

N_LAGS = 7
HORIZON = 1
BATCH_SIZE = 10

mvtseries = pd.read_csv(
    "../assets/daily_multivariate_timeseries.csv",
    parse_dates=["datetime"],
    index_col="datetime",
)

num_vars = mvtseries.shape[1] + 1


def create_training_set(
    data: pd.DataFrame,
    n_lags: int,
    horizon: int,
    test_size: float = 0.2,
    batch_size: int = 16,
):
    data["target"] = data["Incoming Solar"]
    data["time_index"] = np.arange(len(data))
    data["group_id"] = 0  # Assuming a single group for simplicity

    time_indices = data["time_index"].values

    train_indices, _ = train_test_split(
        time_indices, test_size=test_size, shuffle=False
    )

    train_indices, _ = train_test_split(train_indices, test_size=0.1, shuffle=False)

    train_df = data.loc[data["time_index"].isin(train_indices)]
    train_df_mod = train_df.copy()

    target_scaler = StandardScaler()
    target_scaler.fit(train_df_mod[["target"]])
    train_df_mod["target"] = target_scaler.transform(train_df_mod[["target"]])
    train_df_mod = train_df_mod.drop("Incoming Solar", axis=1)

    feature_names = [
        col for col in data.columns if col != "target" and col != "Incoming Solar"
    ]

    training_dataset = TimeSeriesDataSet(
        train_df_mod,
        time_idx="time_index",
        target="target",
        group_ids=["group_id"],
        max_encoder_length=n_lags,
        max_prediction_length=horizon,
        time_varying_unknown_reals=feature_names,
        scalers={name: StandardScaler() for name in feature_names},
    )

    loader = training_dataset.to_dataloader(batch_size=batch_size, shuffle=False)

    return loader

In [5]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, X):
        X = X.view(X.size(0), -1)
        return self.linear(X)

In [6]:
data_loader = create_training_set(
    data=mvtseries, n_lags=N_LAGS, horizon=HORIZON, batch_size=BATCH_SIZE, test_size=0.3
)

model = LinearRegressionModel(N_LAGS * num_vars, HORIZON)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# 3) Training loop
num_epochs = 40
for epoch in range(num_epochs):
    for batch in data_loader:
        x, y = batch

        X = x["encoder_cont"].squeeze(-1)
        y_pred = model(X)
        y_pred = y_pred.squeeze(1)

        y_actual = y[0].squeeze(1)

        loss = criterion(y_pred, y_actual)

        loss.backward()
        optimizer.step()

        optimizer.zero_grad()

    print(f"epoch: {epoch + 1}, loss = {loss.item():.4f}")

epoch: 1, loss = 0.8368
epoch: 2, loss = 0.8122
epoch: 3, loss = 0.7843
epoch: 4, loss = 0.7667
epoch: 5, loss = 0.7594
epoch: 6, loss = 0.7582
epoch: 7, loss = 0.7600
epoch: 8, loss = 0.7632
epoch: 9, loss = 0.7667
epoch: 10, loss = 0.7701
epoch: 11, loss = 0.7733
epoch: 12, loss = 0.7761
epoch: 13, loss = 0.7785
epoch: 14, loss = 0.7807
epoch: 15, loss = 0.7825
epoch: 16, loss = 0.7842
epoch: 17, loss = 0.7856
epoch: 18, loss = 0.7868
epoch: 19, loss = 0.7879
epoch: 20, loss = 0.7888
epoch: 21, loss = 0.7896
epoch: 22, loss = 0.7903
epoch: 23, loss = 0.7909
epoch: 24, loss = 0.7914
epoch: 25, loss = 0.7919
epoch: 26, loss = 0.7923
epoch: 27, loss = 0.7926
epoch: 28, loss = 0.7929
epoch: 29, loss = 0.7931
epoch: 30, loss = 0.7933
epoch: 31, loss = 0.7934
epoch: 32, loss = 0.7935
epoch: 33, loss = 0.7936
epoch: 34, loss = 0.7937
epoch: 35, loss = 0.7937
epoch: 36, loss = 0.7937
epoch: 37, loss = 0.7937
epoch: 38, loss = 0.7937
epoch: 39, loss = 0.7936
epoch: 40, loss = 0.7935
